# DAOS Filesystem with Tensorflow

## Overview

This tutorial shows how to use read and write files on [DAOS Filesystem](https://docs.daos.io/) with TensorFlow, through TensorFlow IO's DAOS file system integration.

A machine running DAOS natively or through a [docker emulator](https://github.com/daos-stack/daos/tree/master/utils/docker) is needed to run this tutorial and/or use the Tensorflow IO DAOS integration. The DAOS Pool and Container IDs should be provided through environmental variables:
```
os.environ['TF_POOL_ID'] = '<pool_id>'
os.environ['TF_CONT_ID'] = '<cont_id>'
```

The pool and container id are part of the filename uri:
```
dfs://<pool_id>/<cont_id>/<path>
```

## Setup and usage

### Install required packages, and restart runtime

In [1]:
try:
  %tensorflow_version 2.x 
except Exception:
  pass

!pip install tensorflow-io

Defaulting to user installation because normal site-packages is not writeable


### Create Pool and Container

In [2]:
%%bash
OUTPUT=$(dmg -i pool create -s 2G)
export TF_POOL_ID=`echo -e $OUTPUT | cut -d':' -f 3 | cut -d ' ' -f 2 | xargs`
OUTPUT=$(daos cont create --pool=$TF_POOL_ID --type=POSIX)
export TF_CONT_ID=`echo -e $OUTPUT | cut -d ' ' -f 4 | xargs`

bash: line 2: dmg: command not found
bash: line 4: daos: command not found


### Read and write files to DAOS with TensorFlow

The following is an example of reading and writing files to DAOS with TensorFlow's API.

It behaves the same way as other file systems (e.g., POSIX or GCS) in TensorFlow once `tensorflow-io` package is imported, as `tensorflow-io` will automatically register `dfs` scheme for use.

The DAOS POOL ID and Container ID should be provided through `TF_POOL_ID` and `TF_CONT_ID` environmental variables.

In [ ]:
import os
import tensorflow as tf
import tensorflow_io as tfio

pool = os.environ['TF_POOL_ID']
cont = os.environ['TF_CONT_ID']

In [ ]:
base = pool + "/" + cont
pathname = 'dfs://{}/dfstest'.format(base)
tf.io.gfile.mkdir(pathname)

filename = pathname + '/hello.txt'
with tf.io.gfile.GFile(filename, mode='w') as w:
  w.write("Hello, world!")

with tf.io.gfile.GFile(filename, mode='r') as r:
  print(r.read())